# Introduction to rio-tiler

The goal of this notebook is to give a quick introduction of the main rio-tiler features.

In [ ]:
%pylab inline

# Requirements

To be able to run this notebook you'll need the following requirements:
- rio-tiler~= 3.0

In [ ]:
# !pip install rio-tiler

In [ ]:
import morecantile
from rio_tiler.io import COGReader
from rio_tiler.profiles import img_profiles
from rio_tiler.models import ImageData

## Data

For this demo we will use some NAIP data hosted on Azure.

https://azure.microsoft.com/fr-fr/services/open-datasets/catalog/naip/


The data is similar to the data hosted on [AWS](https://registry.opendata.aws/naip/), but using the one on Azure is easier because it offers direct `http` access without needing an AWS account.

In [ ]:
# For this DEMO we will use this file
src_path = "https://naipblobs.blob.core.windows.net/naip/v002/al/2019/al_60cm_2019/30087/m_3008701_ne_16_060_20191115.tif"

## rio_tiler.io.COGReader

In `rio-tiler` 2.0 we introduced COGReader, which is a python class providing usefull methods to read and inspect any GDAL/rasterio raster dataset.

Docs: [https://cogeotiff.github.io/rio-tiler/readers/#cogreader](https://cogeotiff.github.io/rio-tiler/readers/#cogreader) 

In [ ]:
?COGReader

### Info

Read GDAL/Rasterio dataset metadata

In [ ]:
# As for Rasterio, using context manager is a good way to 
# make sure the dataset are closed when we exit.
with COGReader(src_path) as cog:
    print("rasterio dataset:")
    print(cog.dataset)
    print()
    print("metadata from rasterio:")
    print(cog.dataset.meta)
    print()
    # Using rio-tiler Info() method
    info = cog.info()
    print("rio-tiler dataset info:")
    print(cog.info().dict(exclude_none=True))

print(cog.dataset.closed)

### Statistics

Return basic data statistics

In [ ]:
with COGReader(src_path) as cog:
    meta = cog.statistics(max_size=256)
    # output from metadata() method is a pydantic model

    assert isinstance(meta, dict)
    print(list(meta))
    print(meta["1"].dict())

#### Plot Histogram values

In [ ]:
fig, axs = plt.subplots(1, 4, sharey=True, tight_layout=True, dpi=150)
# Red (index 1)
axs[0].plot(meta["1"].histogram[1][0:-1], meta["1"].histogram[0])

# Green (index 2)
axs[1].plot(meta["2"].histogram[1][0:-1], meta["2"].histogram[0])

# Blue (index 3)
axs[2].plot(meta["3"].histogram[1][0:-1], meta["3"].histogram[0])

# NIR (index 4)
axs[3].plot(meta["4"].histogram[1][0:-1], meta["4"].histogram[0])

### Preview

Read a low resolution version of the data (useful when working with COG, because this method will only fetch the overview layer it needs)

In [ ]:
with COGReader(src_path) as cog:
    # By default `preview()` will return an array with its longest dimension lower or equal to 1024px
    data = cog.preview()
    print(data.data.shape)
    assert isinstance(data, ImageData)

#### The ImageData class

To ease data manipulation, `rio-tiler` version 2.0 uses a new `ImageData` class that holds the arrays returned by rio-tiler/rasterio low level functions.

Docs: https://cogeotiff.github.io/rio-tiler/models/#imagedata

In [ ]:
print(f"width: {img.width}")
print(f"height: {img.height}")
print(f"bands: {img.count}")
print(f"crs: {img.crs}")
print(f"bounds: {img.bounds}")

print(type(img.data))
print(type(img.mask))

#### Display the data

In [ ]:
# Rasterio doesn't use the same axis order than visualization libraries (e.g matplotlib, PIL)
# in order to display the data we need to change the order (using rasterio.plot.array_to_image).
# the ImageData class wraps the rasterio function in the `data_as_image()` method.
print(type(img))
print(img.data.shape)

image = img.data_as_image()
# data_as_image() returns a numpy.ndarray
print(type(image))
print(image.shape)

# Use only the first 3 bands (RGB)
imshow(image[:,:,0:3])

In [ ]:
# Display NRG image
# The NAIP imagery has 4 bands Red, Green, Blue, NIR
imshow(image[:,:,[3,0,1]])

#### Using Expression

`rio-tiler` reader methods accept `indexes` option to select the bands you want to read, but also `expression` to perform band math.

In [ ]:
with COGReader(src_path) as cog:
    # Return only the last band
    nir_band = cog.preview(indexes=4)
    print(nir_band.data.shape)
    print(nir_band.data.dtype)

In [ ]:
with COGReader(src_path) as cog:
    # Apply NDVI band math
    # (NIR - RED) / (NIR + RED)
    ndvi = cog.preview(expression="(b4-b1)/(b4+b1)")
    print(ndvi.data.shape)
    print(ndvi.data.dtype)
    print("NDVI range: ", ndvi.data.min(), ndvi.data.max())

image = ndvi.post_process(in_range=((-1,1),))
imshow(image.data[0])

### Tile

Read data for a specific slippy map tile coordinates

In [ ]:
with COGReader(src_path) as cog:
    print(f"Bounds in dataset CRS: {cog.bounds}")
    print(f"Bounds WGS84: {cog.geographic_bounds}")
    print(f"MinZoom (WebMercator): {cog.minzoom}")
    print(f"MaxZoom (WebMercator): {cog.maxzoom}")

In [ ]:
# rio-tiler defaults to the WebMercator Grids. The grid definition is provided by the morecantile module
# Docs: https://github.com/developmentseed/morecantile
tms = morecantile.tms.get("WebMercatorQuad")
print(repr(tms))

# Get the list of tiles for the COG minzoom 
with COGReader(src_path) as cog:
    tile_cover = list(tms.tiles(*cog.geographic_bounds, zooms=cog.minzoom))

print(f"Nb of Z{cog.minzoom} Mercator tiles: {len(tile_cover)}")
print(tile_cover)

In [ ]:
with COGReader(src_path) as cog:
    img_1 = cog.tile(*tile_cover[0])
    print(img_1.data.shape)

    img_2 = cog.tile(*tile_cover[1])
    print(img_2.data.shape)

In [ ]:
imshow(img_1.data_as_image()[:,:,0:3])

In [ ]:
imshow(img_2.data_as_image()[:,:,0:3])

In [ ]:
with COGReader(src_path) as cog:
    ndvi = cog.tile(*tile_cover[0], expression="(b4-b1)/(b4+b1)")
    print(ndvi.data.shape)

image = ndvi.post_process(in_range=((-1,1),))
imshow(image.data[0])

### Part 

Read data for a given bounding box

In [ ]:
with COGReader(src_path) as cog:
    # By default `part()` will read the highest resolution. We can limit this by using the `max_size` option.
    img = cog.part((-87.92238235473633, 30.954131465929947, -87.87843704223633, 30.97996389724008), max_size=1024)
    print(img.data.shape)
    print(img.bounds)
    print(img.crs)

In [ ]:
imshow(img.data_as_image()[:,:,0:3])

### Point

Read the pixel value for a specific lon/lat coordinate

In [ ]:
with COGReader(src_path) as cog:
    values = cog.point(-87.92238235473633, 30.954131465929947)
print(values)

### Feature/GeoJSON

Read value for a geojson feature defined area

In [ ]:
feat = {"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-87.91989326477051,30.977388327504983],[-87.92341232299805,30.9747390975502],[-87.92015075683594,30.97282571907513],[-87.91723251342773,30.971869015455276],[-87.9192066192627,30.96914603729001],[-87.92032241821289,30.965466213678003],[-87.91869163513184,30.960093416531947],[-87.91577339172363,30.957885330068873],[-87.91028022766113,30.95700208119036],[-87.90839195251465,30.955971613842785],[-87.91105270385741,30.954646710918635],[-87.91508674621582,30.954793923262105],[-87.92135238647461,30.953321789618688],[-87.92126655578612,30.947506639952913],[-87.92324066162108,30.94353152388283],[-87.9224681854248,30.9393353886492],[-87.92109489440918,30.936832343075928],[-87.92075157165527,30.9326359137502],[-87.91646003723145,30.934034743992026],[-87.91534423828124,30.937494920341457],[-87.91611671447754,30.941764752554082],[-87.91053771972656,30.943973211611713],[-87.91414260864258,30.948242754412334],[-87.91671752929688,30.949862186261473],[-87.91191101074219,30.949494135978654],[-87.90899276733398,30.950377454275596],[-87.90349960327147,30.95045106376507],[-87.90298461914062,30.953174575006617],[-87.89912223815918,30.953763432093723],[-87.89543151855469,30.953027360167685],[-87.89122581481934,30.955529981576515],[-87.89551734924316,30.959651803323208],[-87.89912223815918,30.960903035444577],[-87.90238380432129,30.95979900795299],[-87.90633201599121,30.96053502769875],[-87.9082202911377,30.963479049959364],[-87.91345596313477,30.964877428739207],[-87.91259765625,30.967306143211744],[-87.9085636138916,30.965466213678003],[-87.90547370910643,30.96553981154008],[-87.90667533874512,30.96885165662014],[-87.90684700012207,30.97039714501039],[-87.89517402648926,30.972972903396382],[-87.89328575134277,30.97643166961476],[-87.8957748413086,30.979080852589725],[-87.89852142333984,30.977093972252376],[-87.90006637573242,30.97643166961476],[-87.9019546508789,30.978712914907245],[-87.90633201599121,30.97805062350409],[-87.90461540222168,30.975107050552193],[-87.90521621704102,30.97422396096446],[-87.90796279907227,30.976358080149122],[-87.90976524353026,30.976063721719164],[-87.90907859802245,30.973856004558257],[-87.9111385345459,30.974076778572197],[-87.91379928588867,30.975769362381378],[-87.9177474975586,30.97643166961476],[-87.91929244995116,30.977314738776947],[-87.91989326477051,30.977388327504983]]]}}

In [ ]:
with COGReader(src_path) as cog:
    # we use the feature to define the bounds and the mask
    # but we use `dst_crs` options to keep the projection from the input dataset
    # By default `feature()` will read the highest resolution. We can limit this by using the `max_size` option.
    img = cog.feature(feat, dst_crs=cog.crs, max_size=1024)
    print(img.data.shape)
    print(img.bounds)
    print(img.crs)

In [ ]:
imshow(img.data_as_image()[:,:,0:3])